In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Sampling
dt = 1e-3  # Sampling interval
L = int(1e5)  # Number of samples
t = np.arange(0, L) * dt  # Time axis in ms
fs = 1 / dt  # Sampling frequency in MHz / samples per second
df = fs / L  # Frequency resolution in MHz
f = np.arange(0, L // 2) * df  # Frequency vector

# Carrier and modulation parameters
f0 = 100  # Carrier frequency in MHz
dfm = 0.1  # Frequency spacing for modulation
Nm = 200  # Number of modulating frequencies
ffm = np.arange(1, Nm + 1) * dfm  # FM frequencies
an = np.zeros_like(ffm)  # sine amplitudes
bn = np.zeros_like(ffm)  # cosine amplitudes

# AWG input
an[[int(i / dfm) - 1 for i in [1, 2, 3]]] = [0.3, 0.3, -0.3]
bn[[int(i / dfm) - 1 for i in [1, 2, 3]]] = [0.3, -0.3, 0.3]

# Generate FM waveform / AWG output
FM = np.zeros_like(t)
for nf in range(len(ffm)):
    FM += an[nf] * np.cos(2 * np.pi * ffm[nf] * t) - bn[nf] * np.sin(2 * np.pi * ffm[nf] * t)


# aom simulation (not accurate)
RF = np.cos(2 * np.pi * (f0 * t + FM))


### Time Domain Plots

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(t, FM)
plt.title("FM Modulation Signal")
plt.xlabel("Time (ms)")
plt.ylabel("Amplitude")
plt.xlim([0, 10])
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 3))
plt.plot(t, RF)
plt.title("RF Signal (Carrier with FM)")
plt.xlabel("Time (ms)")
plt.ylabel("Amplitude")
plt.xlim([0, 1])
plt.grid(True)
plt.show()


### Frequency Domain Analysis

In [ ]:
# FFT of FM signal
spekFM = np.fft.fft(FM) / (L / 2)
aspekFM = np.real(spekFM[:L // 2])
bspekFM = np.imag(spekFM[:L // 2])

# --- Combined plot: FM ---
neg_freqs = np.linspace(-5, 0, 100, endpoint=False)  # e.g., 100 points from -5 to just below 0
zero_vals = np.zeros_like(neg_freqs)

# Concatenate with original data
f_ext = np.concatenate((neg_freqs, f))
aspekFM_ext = np.concatenate((zero_vals, aspekFM))
bspekFM_ext = np.concatenate((zero_vals, bspekFM))

# Plot
plt.figure(figsize=(10, 4))
plt.plot(f_ext, aspekFM_ext, label='Real')
plt.plot(f_ext, bspekFM_ext, label='Imag')
plt.title("AWG Input Spectrum")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Amplitude")
plt.legend()
plt.xlim([-5, 5])
plt.grid(True)
plt.show()

# --- Separate plots: FM ---
fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
axs[0].plot(f, aspekFM, label='Real')
axs[0].set_title("FM Signal - Real Part")
axs[0].set_ylabel("Amplitude")
axs[0].legend()
axs[0].grid(True)

axs[1].plot(f, bspekFM, label='Imag', color='orange')
axs[1].set_title("FM Signal - Imaginary Part")
axs[1].set_xlabel("Frequency (MHz)")
axs[1].set_ylabel("Amplitude")
axs[1].legend()
axs[1].grid(True)

plt.xlim([-5, 5])
plt.show()

# FFT of RF signal
spekRF = np.fft.fft(RF) / (L / 2)
aspekRF = np.real(spekRF[:L // 2])
bspekRF = np.imag(spekRF[:L // 2])

# --- Combined plot: RF ---
plt.figure(figsize=(10, 4))
plt.plot(f, aspekRF, label='Real')
plt.plot(f, bspekRF, label='Imag')
plt.title("AOM Output Spectrum")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Amplitude")
plt.legend()
plt.xlim([50, 150])
plt.grid(True)
plt.show()

# --- Separate plots: RF ---
fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
axs[0].plot(f, aspekRF, label='Real')
axs[0].set_title("RF Signal - Real Part")
axs[0].set_ylabel("Amplitude")
axs[0].legend()
axs[0].grid(True)

axs[1].plot(f, bspekRF, label='Imag', color='orange')
axs[1].set_title("RF Signal - Imaginary Part")
axs[1].set_xlabel("Frequency (MHz)")
axs[1].set_ylabel("Amplitude")
axs[1].legend()
axs[1].grid(True)

plt.xlim([50, 150])
plt.show()

# Example frequency array
f = np.linspace(50, 150, 500)  # 500 points from 50 to 150 MHz

# Create zero arrays for real and imaginary parts
a = np.zeros_like(f)
b = np.zeros_like(f)

# Define desired frequencies and amplitudes
target_real_freqs = [80, 100, 120]
target_imag_freqs = [90, 110]

# Set amplitudes at the indices closest to target frequencies
for freq in target_real_freqs:
    idx = (np.abs(f - freq)).argmin()
    a[idx] = 0.3

for freq in target_imag_freqs:
    idx = (np.abs(f - freq)).argmin()
    b[idx] = -0.3

# Plot
plt.figure(figsize=(10, 4))
plt.plot(f, a, label='Real')
plt.plot(f, b, label='Imag')
plt.title("Desired AOM Output Spectrum")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Amplitude")
plt.legend()
plt.xlim([50, 150])
plt.grid(True)
plt.show()